<div style="text-align: center;">
    <h1><strong>Single-Image Classifier</strong></h1>
</div>

#### Imports

In [1]:
import torch
import torchvision
from torchvision import models
from torchvision import transforms
from PIL import Image

print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")
device = "cuda" if torch.cuda.is_available() else "cpu"
gpu = torch.cuda.get_device_name(0)
print(f"GPU: {gpu}")

PyTorch version: 2.5.1+cu124
torchvision version: 0.20.1+cu124
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


#### Define Paths

In [44]:
WEIGHTS_PATH = r"models/DenseNet_True_97.41_weights.pt"
IMAGE_PATH = r"test-imagery/screenshot0.png"

#### Define Colors

In [15]:
RED = "\033[91m"
GREEN = "\033[92m"
BLUE = "\033[94m"
YELLOW = "\033[93m"
CYAN = "\033[96m"
RESET = "\033[0m"

#### Apply weights

In [3]:
model = torchvision.models.densenet169(weights=None).to(device)

model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=model.classifier.in_features,
                    out_features=10,
                    bias=True)
                    ).to(device)

model.load_state_dict(torch.load(WEIGHTS_PATH))
model.to(device)
model.eval()

C:\Users\aquas\AppData\Local\Temp\ipykernel_11948\458335799.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(WEIGHTS_PATH))


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

#### Define transformation and label map

In [4]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3444, 0.3803, 0.4078], std=[0.2027, 0.1369, 0.1155])
])

In [5]:
label_map = {
    "AnnualCrop": 0, "Forest": 1, "HerbaceousVegetation": 2, "Highway": 3, "Industrial": 4,
    "Pasture": 5, "PermanentCrop": 6, "Residential": 7, "River": 8, "SeaLake": 9
}

idx_to_class = {v: k for k, v in label_map.items()}

#### Define classifier function

In [6]:
def classify(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return idx_to_class[predicted.item()]

#### Classify an image

In [45]:
prediction = classify(IMAGE_PATH, model, transform, device)
print(f"Predicted class: {GREEN}{prediction}{RESET}")

Predicted class: Residential
